# Convert Spacy Model to CoreML
Google Search "Spacy export model to coreml python"

## References
- [CoreML Tools](https://apple.github.io/coremltools/docs-guides/)
- [CoreML Tools API](https://apple.github.io/coremltools/index.html)
- [Converting Models to Core ML](https://huggingface.co/blog/fguzman82/frompytorch-to-coreml)
- [PyTorch to CoreML Conversion](https://coremltools.readme.io/v6.3/docs/pytorch-conversion)

### Imports

In [ ]:
import spacy
import torch
import torch.nn as nn

### Load the spaCy Model

In [ ]:
nlp = spacy.load('./training/ner_and_textcat')

### Wrap the spaCy model in a PyTorch Model

In [ ]:
# Extract word vectors
word_vectors = torch.FloatTensor(nlp.vocab.vectors.data)

# Define a PyTorch model
class PyTorchWrapper(nn.Module):
    def __init__(self, vocab_size, embedding_dim, pretrained_embeddings):
        super(PyTorchWrapper, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim).from_pretrained(pretrained_embeddings)
        # Add other layers as needed, e.g., LSTM, Linear, etc.
        
    def forward(self, x):
        embedded = self.embedding(x)
        # Pass through other layers
        return embedded
    
# Instantiate the model
vocab_size, embedding_dim = word_vectors.shape
model = PyTorchWrapper(vocab_size, embedding_dim, word_vectors)
model

### Script the Model

In [ ]:
from coreml import (
    script_model,
    convert_to_coreml
)
scripted = script_model(model)
scripted

### Convert the scripted model to CoreML

In [ ]:
coreml_model = convert_to_coreml(scripted, (1, 1))
coreml_model

### Save the model

In [ ]:
coreml_model.save("construction.mlpackage")